In [ ]:
import numpy as np 
import pandas as pd 

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/spam-mails-dataset/spam_ham_dataset.csv

READING THE DATA IN THE CSV FILE 

In [ ]:
data = pd.read_csv('../input/spam-mails-dataset/spam_ham_dataset.csv')
data

In [ ]:
data = pd.read_csv('../input/spam-mails-dataset/spam_ham_dataset.csv').dropna()
data.head()

IMPORTING THE REQUIRED LIBRARIES 

In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
import re
import nltk
import time
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import math
import random
from nltk.stem import WordNetLemmatizer

nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!

USING BINARY NAIVE BAYE'S THEOREM TO CREATE A SPAM AND HAM FILTER AND MAKING THE DATA CLASSIFIED AND CALCULATING THE CONFUSION MATRIX IN WHICH THE TRUE POSITIVE , TRUE NEGATIVE , FALSE POSITIVE , FALSE NEGATIVE VALUES ARE OBTAINED THROUGH WHICH CALCULATING THE ACCURACY , SPECIFICITY , SENSITIVITY VALUES THE  dATAFRAME  WILL  BE SPLIT INTO  TRAIN  AND TEST. THE TRAINING DATASET WILL BE USED TO BUILD THE MODEL

In [ ]:
class BinaryNaiveBayes():
        
    def vectorize(self, data, splitRatio):
        arr = data['text'].to_list()
        y = data['label_num'].to_list()
        
        temp = list(zip(arr, y))
        random.shuffle(temp)
        arr, y = zip(*temp)
        
        x = self.vectorizer.fit_transform(arr).toarray()
        train_x = x[:int(len(x)*splitRatio)]
        test_x = x[int(len(x)*splitRatio):]
        
        train_y = y[:int(len(y)*splitRatio)]
        test_y = y[int(len(y)*splitRatio):]
        return train_x, train_y, test_x, test_y
    
    def count(self):
        for i in range(len(self.labels)):
            currX = self.data[i]
            label = self.labels[i]
            if(label == 0):
                for i in range(len(currX)):
                    self.positiveHashMap[i] += currX[i]
                self.numPositives += 1
            else:
                for i in range(len(currX)):
                    self.negativeHashMap[i] += currX[i]
                self.numNegatives += 1
    
    def __init__(self, data):
        self.numFeatures = 20000
        self.vectorizer = CountVectorizer(max_features=self.numFeatures)
        self.data, self.labels, self.test_data, self.test_labels = self.vectorize(data, 0.8) 
        self.positiveHashMap = [0 for i in range(self.numFeatures)]
        self.numPositives = 0
        self.negativeHashMap = [0 for i in range(self.numFeatures)]
        self.numNegatives = 0
        self.test_matrix = [[0,0],[0,0]]
        
        self.count()
        self.posSum = sum(self.positiveHashMap)
        self.negSum = sum(self.negativeHashMap)
        self.lemma = WordNetLemmatizer()
    
    def predict(self, document):
        """print(self.numPositives, self.numNegatives)
        print(self.posSum, self.negSum)
        print(self.positiveHashMap)
        print(self.negativeHashMap)"""
        priorPos = self.numPositives/(self.numPositives+self.numNegatives)
        likelihoodPos = math.log(priorPos)
        priorNeg = self.numNegatives/(self.numPositives+self.numNegatives)
        likelihoodNeg = math.log(priorNeg)
        for i in range(self.numFeatures):
            if document[i] == 0:
                continue
            #print(likelihoodPos, likelihoodNeg)
            likelihoodPos += math.log(self.prob(i, 0))
            likelihoodNeg += math.log(self.prob(i, 1))
            
        #print(math.exp(likelihoodPos), math.exp(likelihoodNeg))
        if(likelihoodPos > likelihoodNeg):
            return 0
        return 1
    
    def prob(self, i, c):
        if(c == 0):
            return (self.positiveHashMap[i]+1)/(self.posSum + self.numFeatures)
        return (self.negativeHashMap[i]+1)/(self.negSum + self.numFeatures)
    
    def test(self):
        for i in range(len(self.test_data)):
            prediction = self.predict(self.test_data[i])
            label = self.test_labels[i]
            self.test_matrix[prediction][label] += 1
      
    def Sensitivity(self):
        return self.test_matrix[1][1]/(self.test_matrix[1][1] + self.test_matrix[1][0])
    
    def Specificity(self):
        return self.test_matrix[0][0]/(self.test_matrix[0][0] + self.test_matrix[0][1])
    
    def accuracy(self):
        return (self.test_matrix[0][0]+self.test_matrix[1][1])/(sum(self.test_matrix[0]) + sum(self.test_matrix[1]))
    
    def text_to_vector(self, text):
        text = re.sub(r"http\S+", "", text)
        pattern = "[^a-zA-Z0-9]"
        text = re.sub(pattern," ",text)
        text = text.lower()
        text = nltk.word_tokenize(text)
        text = [self.lemma.lemmatize(word) for word in text]
        stopwords = nltk.corpus.stopwords.words("english")
        text = [word for word in text if word not in stopwords]
        text = " ".join(text)
        return self.vectorizer.transform([text]).toarray()
    

CALCULATING THE ACCURACY

In [ ]:
nb = BinaryNaiveBayes(data)
nb.test()
nb.accuracy()

In [ ]:
nb.test_matrix

CALCULATING THE SPECIFICITY

In [ ]:
nb = BinaryNaiveBayes(data)
nb.test()
nb.Specificity()

CALCULATING THE SENSITIVITY

In [ ]:
nb = BinaryNaiveBayes(data)
nb.test()
nb.Sensitivity()

THE CORRECT CLASSIFICATION AND MISCLASSIFICATION ARE DETERMINED BY THE CONFUSION MATRIX
CREATING A FUNCTION FOR CONFUSION MATRIX AND THE LAYOUT OF THE CONFUSION MATRIX AND THE COLOURS THAT ARE GIVE TO THE VALUES OF TN,TP,FN,FP AND SETTING THE LABELS AND TITLE TO THE MATRIX

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

def make_confusion_matrix(cf,
                          group_names=None,
                          categories='auto',
                          count=True,
                          percent=True,
                          cbar=True,
                          xyticks=True,
                          xyplotlabels=True,
                          sum_stats=True,
                          figsize=None,
                          cmap='Blues',
                          title=None):
    '''
    This function will make a pretty plot of an sklearn Confusion Matrix cm using a Seaborn heatmap visualization.
    Arguments
    ---------
    cf:            confusion matrix to be passed in
    group_names:   List of strings that represent the labels row by row to be shown in each square.
    categories:    List of strings containing the categories to be displayed on the x,y axis. Default is 'auto'
    count:         If True, show the raw number in the confusion matrix. Default is True.
    normalize:     If True, show the proportions for each category. Default is True.
    cbar:          If True, show the color bar. The cbar values are based off the values in the confusion matrix.
                   Default is True.
    xyticks:       If True, show x and y ticks. Default is True.
    xyplotlabels:  If True, show 'True Label' and 'Predicted Label' on the figure. Default is True.
    sum_stats:     If True, display summary statistics below the figure. Default is True.
    figsize:       Tuple representing the figure size. Default will be the matplotlib rcParams value.
    cmap:          Colormap of the values displayed from matplotlib.pyplot.cm. Default is 'Blues'
                   See http://matplotlib.org/examples/color/colormaps_reference.html
                   
    title:         Title for the heatmap. Default is None.
    '''


    # CODE TO GENERATE TEXT INSIDE EACH SQUARE
    blanks = ['' for i in range(cf.size)]

    if group_names and len(group_names)==cf.size:
        group_labels = ["{}\n".format(value) for value in group_names]
    else:
        group_labels = blanks

    if count:
        group_counts = ["{0:0.0f}\n".format(value) for value in cf.flatten()]
    else:
        group_counts = blanks

    if percent:
        group_percentages = ["{0:.2%}".format(value) for value in cf.flatten()/np.sum(cf)]
    else:
        group_percentages = blanks

    box_labels = [f"{v1}{v2}{v3}".strip() for v1, v2, v3 in zip(group_labels,group_counts,group_percentages)]
    box_labels = np.asarray(box_labels).reshape(cf.shape[0],cf.shape[1])


    # CODE TO GENERATE SUMMARY STATISTICS & TEXT FOR SUMMARY STATS
    if sum_stats:
        #Accuracy is sum of diagonal divided by total observations
        accuracy  = np.trace(cf) / float(np.sum(cf))

        #if it is a binary confusion matrix, show some more stats
        if len(cf)==2:
            #Metrics for Binary Confusion Matrices
            Sensitivity = cf[1,1] / (cf[1,1]+cf[1,0])
            Specificity    = cf[0,0] / (cf[0,0]+cf[0,1])
            stats_text = "\n\nAccuracy={:0.3f}\nSensitivity={:0.3f}\nSpecificity={:0.3f}".format(
                accuracy,Sensitivity,Specificity)
        else:
            stats_text = "\n\nAccuracy={:0.3f}".format(accuracy)
    else:
        stats_text = ""


    # SET FIGURE PARAMETERS ACCORDING TO OTHER ARGUMENTS
    if figsize==None:
        #Get default figure size if not set
        figsize = plt.rcParams.get('figure.figsize')

    if xyticks==False:
        #Do not show categories if xyticks is False
        categories=False


    # MAKE THE HEATMAP VISUALIZATION
    plt.figure(figsize=figsize)
    sns.heatmap(cf,annot=box_labels,fmt="",cmap=cmap,cbar=cbar,xticklabels=categories,yticklabels=categories)

    if xyplotlabels:
        plt.ylabel('True label')
        plt.xlabel('Predicted label' + stats_text)
    else:
        plt.xlabel(stats_text)
    
    if title:
        plt.title(title)

PRINTING THE CONFUSION MATRIX

In [ ]:
make_confusion_matrix(np.array(nb.test_matrix), categories = ['Not Spam', 'Spam'], title = 'Confusion Matrix for Naive Bayes Spam Prediction Algorithm')

PRINTING THE TOTAL VALUES OF SPAM AND HAM IN THE GIVEN DATASET TO THE IN-DEPTH IDEA OF OUR GIVEN DATASET

In [ ]:
data.label_num.value_counts()

FINDING THE LENGTH OF EVERY SPAM OR HAM MESSAGE TO HELP CATEGORIZE THEM BETTER

In [ ]:
data['length'] = data['text'].apply(len)
data.head()

TRAINING OUR MODEL USING THE SKLEARN LIBRARY

In [ ]:
X = data['text']
Y = data['label_num']
from sklearn.model_selection import train_test_split as tt
X_train, X_test, Y_train, Y_test = tt(X, Y,test_size=0.2, random_state=100)

FINDING THE XTEST AND TRAIN SHAPE FOR THE SPLIT DATA

In [ ]:
X_train.shape

In [ ]:
X_test.shape

WORDS THAT ARE NOT IN OUR DICTIONARY ARE EXCLUDED FROM OUR DATA AS THEY CAUSE UNNECESSARY ERRORS.FOR THIS , WE USE SKLEARN FEATURE

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vector = CountVectorizer(stop_words ='english')
vector.fit(X_train)

PRINTING THE FREQUENCY OF WORDS THAT ARE BEING USED

In [ ]:
vector.vocabulary_

TRAINING THE MODEL WITH THE FINAL VECTOR INPUTS AND PROCEED WITH THE PREDICTIONS

In [ ]:
X_train_transformed =vector.transform(X_train)
X_test_transformed =vector.transform(X_test)

BULDING THE FINAL MODEL USING THE MULTINOMIAL NAIVE BAYES LIBRARIES FROM SKLEARN

In [ ]:
from sklearn.naive_bayes import MultinomialNB
model = MultinomialNB()
model.fit(X_train_transformed,Y_train)
y_pred = model.predict(X_test_transformed)
y_pred_prob = model.predict_proba(X_test_transformed)

PERFORMING THE PREDICTIONS - TO HELP THE MODEL PREDICT,WE IMPORT THE CONFUSION MATRIX,ACCURACY SCORE

In [ ]:
from sklearn.metrics import confusion_matrix,accuracy_score
print(confusion_matrix(Y_test,y_pred))
print()
print(accuracy_score(Y_test,y_pred))

PRINTING THE CONFUSION MATRIX USING MULLTIBIONOMIAL NAIVES BAYES THEOREM

In [ ]:
 from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
confusion_matrix = metrics.confusion_matrix(Y_test,y_pred)
cm_display = metrics.ConfusionMatrixDisplay(confusion_matrix=confusion_matrix)
cm_display.plot()
plt.show()

In [ ]:
cm = confusion_matrix(Y_test,y_pred)
cm

CALCULATING THE SENSITIVITY FROM THE CONFUSION MATRIX

In [ ]:
from sklearn.metrics import confusion_matrix
y_true = [0, 0, 0, 1, 1, 1, 1, 1]
y_pred = [0, 1, 0, 1, 0, 1, 0, 1]
tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
sensitivity= tp / (fn+tp)
sensitivity

CALCULATING THE SPECIFICITY FROM THE CONFUSION MATRIX

In [ ]:
from sklearn.metrics import confusion_matrix
y_true = [0, 0, 0, 1, 1, 1, 1, 1]
y_pred = [0, 1, 0, 1, 0, 1, 0, 1]
tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
specificity= tn / (tn+fp)
specificity

IMPORTING THE ROC CURVE FEATURE FROM SKLEARN TO FIND TRUE AND FALSE POSITIVE RATES

In [ ]:
from sklearn.metrics import roc_curve, auc

false_positive_rate, true_positive_rate, thresholds = roc_curve(Y_test, y_pred_prob[:,1])
roc_auc = auc(false_positive_rate, true_positive_rate)
print(roc_auc)

FROM THE CONFUSION MATRIX ACCESSING THE TP,FP,FN,TN VALUES

In [ ]:
TP = cm[1][1]
FP = cm[0][1]
FN = cm[1][0]
TN = cm[0][0]

PRINTING TPR,FPR,THRESHOLDS

In [ ]:
print(false_positive_rate)
print()
print(true_positive_rate)
print()
print(thresholds)

ORGANISING THE TPR,FPR,THRESHOLD VALUES IN A DATAFRAME

In [ ]:
df = pd.DataFrame({'Threshold': thresholds, 
              'TPR': true_positive_rate, 
              'FPR':false_positive_rate
             })
df.head()

CALCULATING THE SPEACIFICITY THAT IS 1-FPR

In [ ]:
from sklearn.metrics import roc_curve, auc

false_positive_rate, true_positive_rate, thresholds = roc_curve(Y_test, y_pred_prob[:,1])
specificity = 1-false_positive_rate
roc_auc = auc(false_positive_rate, true_positive_rate)
roc_auc


CALCULATING THE SENSITITY THAT IS TPR

In [ ]:
from sklearn.metrics import roc_curve, auc

false_positive_rate, true_positive_rate, thresholds = roc_curve(Y_test, y_pred_prob[:,1])
sensitivity = true_positive_rate
roc_auc = auc(false_positive_rate, true_positive_rate)
roc_auc

PLOTTING THE GRAPH AGAINST SENSITIVITY AND THRESHOLD

In [ ]:
%matplotlib inline  
plt.ylabel('sensitivity')     
plt.xlabel('threshold')
plt.title('ROC')
plt.plot(thresholds, sensitivity)


In [ ]:
from sklearn.metrics import roc_curve, auc

false_positive_rate, true_positive_rate, thresholds = roc_curve(Y_test, y_pred_prob[:,1])
specificity = 1-false_positive_rate
roc_auc = auc(false_positive_rate, true_positive_rate)
roc_auc


PLOTTING THE GARPH AGAINST SPECIFICITY AND THRESHOLD

In [ ]:
%matplotlib inline  
plt.ylabel('specificity')     
plt.xlabel('threshold')
plt.title('ROC')
plt.plot(thresholds, specificity)

CALCULATING THE PREVELENCE VALUE FROM THE CONFUSION MATRIX

In [ ]:
prevelence = (FN+TP)/(FN+TP+FP+TN)
prevelence

CALCULATING THE ACCURACY USING TPR,FPR AND PREVELENCE

In [ ]:
from sklearn.metrics import roc_curve, auc
false_positive_rate, true_positive_rate, thresholds = roc_curve(Y_test, y_pred_prob[:,1])
specificity = 1-false_positive_rate
sensitivity = true_positive_rate
accuracy = (true_positive_rate)*prevelence + specificity*(1-prevelence)
roc_auc = auc(false_positive_rate, true_positive_rate)
roc_auc


POLTTING AGAINST ACCURACY AND THRESHOLD

In [ ]:
%matplotlib inline  
plt.ylabel('accuracy')     
plt.xlabel('threshold')
plt.title('ROC')
plt.plot(thresholds, accuracy)

PLOTTING SPECIFICITY,SENSITIVITY,ACCURACY AGAINST THRESHOLD

In [ ]:
%matplotlib inline  
plt.figure(figsize=(6,6),dpi=100)
plt.ylabel('accuracy / specificity / sensitivity')
plt.xlabel('Threshold')
plt.title('ROC (accuracy vs threshold)')
plt.plot(thresholds , accuracy , color = 'g', linestyle='-',label='accuracy')
plt.plot(thresholds , specificity , color = 'r', linestyle='-',label='specificity')
plt.plot(thresholds , sensitivity , color = 'b', linestyle='-',label='sensitivity')

In [ ]:
vectorizer=CountVectorizer()
x=vectorizer.fit_transform(data['text'])
print(x[:2])

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import statistics
import numpy as  np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import confusion_matrix
from sklearn import metrics
y=data['label_num']
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.33)
print(x_test)
print(x_train)
data['label_num']
bayes=MultinomialNB()
bayes.fit(x_train,y_train)
y_predict=bayes.predict(x_test)


CALCULATING THE MEAN OF CLASSIFIED DATA AND PLOTTING IT IN BARCHART FORM TO COMPARE IT WITH UNCLASSIFIED DATA

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import statistics
import numpy as  np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import confusion_matrix
from sklearn import metrics
mean=statistics.mean(data['label_num'])
mode=statistics.mode(data['label_num'])
mode=str(mode)
print("mean of the original data is: ",mean)
print("mode of the original data is: ",mode.replace('0','ham'))
d1={
    "mean_of_classified_data":"mean",
    "mean":mean
}
df2=pd.DataFrame(d1,index=[0])
df2
df2.plot(x="mean_of_classified_data",y="mean",kind="bar")
plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import statistics
import numpy as  np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import confusion_matrix
from sklearn import metrics
mean=statistics.mean(data['label_num'])
mode=statistics.mode(data['label_num'])
mode=str(mode)
wrongly_classified=[]
for x,y in zip(y_predict,y_test):
  if x != y:
    print(x,y)
    wrongly_classified.append(x)
   
wrongly_classified

CALCULATING THE MEAN OF UNCLASSIFIED DATA AND PLOTTING IT IN BARCHART FORM TO COMPARE IT WITH CLASSIFIED DATA

In [ ]:
s = np.sum(wrongly_classified)
mean1 = s / (len(wrongly_classified))
d={
    "mean_of_wrongly_classified_data":"mean",
    "mean":mean1
}
df4=pd.DataFrame(d,index=[0])
df4.plot(x="mean_of_wrongly_classified_data",y="mean",kind="bar")
plt.show()